In [1]:
!pip install pymysql
!pip install xgboost
import numpy as np
import pandas as pd
import datetime
import os
import warnings 
warnings.filterwarnings('ignore')
os.chdir('/home/ec2-user/SageMaker/daemon_branch/bonzo_ds_models/')
from models.ml_utils import *
from db_utils import *
import traceback

     |████████████████████████████████| 43 kB 458 kB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 173.5 MB 22 kB/s s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Inside init_config
Initialized logger path:  /home/ec2-user/SageMaker/daemon_branch/bonzo_ds_models


In [2]:
%%time
con = db_connection()
weekly_df = pd.read_sql_query("""select * from morphe_staging.fact_sales_ending_inventory_sku_by_week""", con=con)

connecting to db...
connected to db
CPU times: user 37.5 s, sys: 1.37 s, total: 38.8 s
Wall time: 38.9 s


In [3]:
%%time
daily_df = pd.read_sql_query("""select country,channel,sub_channel,gross_revenue,sales_date,sku from morphe_staging.sales_ending_inventory_sku_by_date""", con=con)
con.close()

CPU times: user 3min 26s, sys: 12 s, total: 3min 39s
Wall time: 3min 38s


In [213]:
daily_df1=daily_df.copy()
# daily_df1['country']=daily_df1['country'].replace('AUS','AU').replace('CAN','CA').replace('CN','CA')

In [137]:
# daily_df1['sub_channel'].unique()

array(['US', 'AU', 'CA', 'EU', 'JH', 'PL', 'UK',
       'Wholesale : Independent Accoun', 'Wholesale : Ulta',
       'Wholesale : UK - Wholesale', 'Wholesale : CAN - Wholesale',
       'Wholesale : Operations- Wholes', 'Wholesale : Asia - Wholesale',
       'Wholesale : AUS - Wholesale', 'Wholesale : MEX - Wholesale',
       'Wholesale : Sephora - US', 'Victoria Gardens', 'Del Amo', 'Brea',
       'Valley Fair', 'Liverpool One', 'Galleria Dallas',
       'Mall Of America', 'Fashion Valley', 'Cherry Hill Mall', 'Burbank',
       'Roosevelt Field', 'Square One Shopping Centre', 'Arden Fair',
       'International Plaza', 'Intu Victoria Centre', 'Fresno',
       'Houston Galleria', 'Los Cerritos', 'Tysons Corner',
       'Melbourne Central', 'Southpark Mall', 'West Edmonton',
       'Washington Square Mall', 'Lenox Square', 'Memorial City Mall',
       'Woodfield', 'Birmingham', 'Barton Creek Square', 'Scottsdale',
       'King Of Prussia', 'Garden State', 'Florida Mall',
       'Westfiel

In [138]:
# daily_df1.groupby('sales_date').get_group('PL')

,country,channel,sub_channel,gross_revenue,sales_date,sku
380,None,Ecomm,PL,38.0,2020-03-04,100017
381,None,Ecomm,PL,160.0,2020-03-04,100079
382,None,Ecomm,PL,114.0,2020-03-04,100005
902,None,Ecomm,PL,72.0,2020-08-22,100065
1479,None,Ecomm,PL,72.0,2020-03-05,100025
...,...,...,...,...,...,...
14756734,None,Ecomm,PL,168.0,2021-12-11,100019
14756735,None,Ecomm,PL,1344.0,2021-12-11,100050
14756736,None,Ecomm,PL,576.0,2021-12-11,100051
14756737,None,Ecomm,PL,252.0,2021-12-11,100059


## Monthly analysis of contribution of different subchannels towards total sales


In [214]:
df_axes=pd.DataFrame()
df_axes['sales_date']=daily_df['sales_date']
df_axes['sales_date'] = pd.to_datetime(df_axes['sales_date'],errors='coerce')
df_axes.set_index('sales_date',inplace=True)
df_axes=df_axes.resample('m').count()
df_axes.reset_index(inplace=True)
df_axes

In [215]:
daily_df1=daily_df1.groupby('channel').get_group('Ecomm')
daily_df1['sales_date'] = pd.to_datetime(daily_df1['sales_date'],errors='coerce')
# daily_df1.set_index('sales_date',inplace=True)

In [216]:
abz=daily_df1[(daily_df1['sales_date'] > '2020-12-31') & (daily_df1['sales_date'] < '2021-12-11')]
abz.groupby('sub_channel').get_group('PL')

,country,channel,sub_channel,gross_revenue,sales_date,sku
80704,None,Ecomm,PL,28.0,2021-01-31,100003
80705,None,Ecomm,PL,38.0,2021-01-31,100016
87487,None,Ecomm,PL,64.0,2021-02-01,100082
87488,None,Ecomm,PL,84.0,2021-02-01,100003
87489,None,Ecomm,PL,24.0,2021-02-01,100006
...,...,...,...,...,...,...
14735356,None,Ecomm,PL,28.0,2021-12-10,100003
14735357,None,Ecomm,PL,38.0,2021-12-10,100005
14735358,None,Ecomm,PL,114.0,2021-12-10,100016
14735359,None,Ecomm,PL,24.0,2021-12-10,100019


In [30]:
# daily_df1.sub_channel.unique()

array(['US', 'AU', 'CA', 'EU', 'JH', 'PL', 'UK', 'BadHabit', 'REM'],
      dtype=object)

In [217]:
daily_df1.set_index('sales_date',inplace=True)

In [218]:
sku_number=str(input("enter the sku number: "))
daily_df1=daily_df1.groupby('sku').get_group(sku_number)
x=daily_df1.sub_channel.unique()

enter the sku number: 100016


In [219]:
if 'US' in x:
    df_US=daily_df1.groupby('sub_channel').get_group('US')
    df_US=df_US['gross_revenue'].resample('m').sum()
    df_US=df_US.to_frame()
    df_US.rename(columns={'gross_revenue':'US_sales'},inplace=True)
else:
    df_US=pd.DataFrame()

if 'PL' in x:
    df_PL=daily_df1.groupby('sub_channel').get_group('PL')
    df_PL=df_PL['gross_revenue'].resample('m').sum()
    df_PL=df_PL.to_frame()
    df_PL.rename(columns={'gross_revenue':'PL_sales'},inplace=True)
else:
    df_PL=pd.DataFrame()

if 'JH' in x:
    df_JH=daily_df1.groupby('sub_channel').get_group('JH')
    df_JH=df_JH['gross_revenue'].resample('m').sum()
    df_JH=df_JH.to_frame()
    df_JH.rename(columns={'gross_revenue':'JH_sales'},inplace=True)
else:
    df_JH=pd.DataFrame()

if 'BadHabit' in x:
    df_BadHabit=daily_df1.groupby('sub_channel').get_group('BadHabit')
    df_BadHabit=df_BadHabit['gross_revenue'].resample('m').sum()
    df_BadHabit=df_BadHabit.to_frame()
    df_BadHabit.rename(columns={'gross_revenue':'Badhabit_sales'},inplace=True)
else:
    df_BadHabit=pd.DataFrame()

if 'REM' in x:
    df_REM=daily_df1.groupby('sub_channel').get_group('REM')
    df_REM=df_REM['gross_revenue'].resample('m').sum()
    df_REM=df_REM.to_frame()
    df_REM.rename(columns={'gross_revenue':'REM_sales'},inplace=True)
else:
    df_REM=pd.DataFrame()


In [220]:
if df_PL.empty:
    df_PL['PL_sales']=np.NAN
    df_PL['sales_date']=df_axes['sales_date']
    df_PL.set_index('sales_date',inplace=True)

    
else:pass

if df_US.empty:
    df_US['US_sales']=np.NAN
    df_US['sales_date']=df_axes['sales_date']
    df_US.set_index('sales_date',inplace=True)

else:pass

if df_BadHabit.empty:
    df_BadHabit['BadHabit_sales']=np.NAN
    df_BadHabit['sales_date']=df_axes['sales_date']
    df_BadHabit.set_index('sales_date',inplace=True)

else:pass

if df_REM.empty:
    df_REM['REM_sales']=np.NAN
    df_REM['sales_date']=df_axes['sales_date']
    df_REM.set_index('sales_date',inplace=True)
else:pass

if df_JH.empty:
    df_JH['JH_sales']=np.NAN
    df_JH['sales_date']=df_axes['sales_date']
    df_JH.set_index('sales_date',inplace=True)
else:pass

In [221]:
try:
    df_extended=pd.merge(df_BadHabit,df_REM,on='sales_date',how='outer')
    df_extended=pd.merge(df_extended,df_JH,on='sales_date',how='outer')
    df_extended=pd.merge(df_extended,df_PL,on='sales_date',how='outer')
    df_extended=pd.merge(df_extended,df_US,on='sales_date',how='outer')
except:
    print(traceback.print.exc())

In [222]:
df_extended["sum"] = df_extended.sum(axis=1)
df_percentage=df_extended.copy()
# df_extended

In [223]:
if 'BadHabit_sales' in df_percentage.columns:
    df_percentage['BadHabit_sales']=(df_percentage['BadHabit_sales']/df_percentage['sum'])*100
if 'REM_sales' in df_percentage.columns:
    df_percentage['REM_sales']=(df_percentage['REM_sales']/df_percentage['sum'])*100
if 'JH_sales' in df_percentage.columns:
    df_percentage['JH_sales']=(df_percentage['JH_sales']/df_percentage['sum'])*100
if 'PL_sales' in df_percentage.columns:
    df_percentage['PL_sales']=(df_percentage['PL_sales']/df_percentage['sum'])*100
if 'US_sales' in df_percentage.columns:
    df_percentage['US_sales']=(df_percentage['US_sales']/df_percentage['sum'])*100
    
df_percentage['sum']=(df_percentage['sum']/df_percentage['sum'])*100
    


In [224]:
df_percentage.replace(np.NaN,0,inplace=True)
display(f"Monthly sales for sku number {sku_number} is :", df_percentage)

'Monthly sales for sku number 100016 is :'

,BadHabit_sales,REM_sales,JH_sales,PL_sales,US_sales,sum
sales_date,,,,,,
2017-01-31,0.0,0.0,0.0,0.000000,0.000000,0.0
2017-02-28,0.0,0.0,0.0,0.000000,0.000000,0.0
2017-03-31,0.0,0.0,0.0,0.000000,0.000000,0.0
2017-04-30,0.0,0.0,0.0,0.000000,0.000000,0.0
2017-05-31,0.0,0.0,0.0,0.000000,0.000000,0.0
2017-06-30,0.0,0.0,0.0,0.000000,0.000000,0.0
2017-07-31,0.0,0.0,0.0,0.000000,0.000000,0.0
2017-08-31,0.0,0.0,0.0,0.000000,0.000000,0.0
2017-09-30,0.0,0.0,0.0,0.000000,0.000000,0.0


In [225]:
df_percentage.to_csv(fr'/home/ec2-user/SageMaker/Pratik/Daily Forecast/{sku_number} sku sub channel proportion by month.csv')

## Yearly analysis of contribution of different subchannels towards total sales

In [226]:
df_axes=pd.DataFrame()
df_axes['sales_date']=daily_df['sales_date']
df_axes['sales_date'] = pd.to_datetime(df_axes['sales_date'],errors='coerce')
df_axes.set_index('sales_date',inplace=True)
df_axes=df_axes.resample('y').count()
df_axes.reset_index(inplace=True)
df_axes

,sales_date
0,2017-12-31
1,2018-12-31
2,2019-12-31
3,2020-12-31
4,2021-12-31


In [227]:
if 'US' in x:
    df_US=daily_df1.groupby('sub_channel').get_group('US')
    df_US=df_US['gross_revenue'].resample('y').sum()
    df_US=df_US.to_frame()
    df_US.rename(columns={'gross_revenue':'US_sales'},inplace=True)
else:
    df_US=pd.DataFrame()

if 'PL' in x:
    df_PL=daily_df1.groupby('sub_channel').get_group('PL')
    df_PL=df_PL['gross_revenue'].resample('y').sum()
    df_PL=df_PL.to_frame()
    df_PL.rename(columns={'gross_revenue':'PL_sales'},inplace=True)
else:
    df_PL=pd.DataFrame()

if 'JH' in x:
    df_JH=daily_df1.groupby('sub_channel').get_group('JH')
    df_JH=df_JH['gross_revenue'].resample('y').sum()
    df_JH=df_JH.to_frame()
    df_JH.rename(columns={'gross_revenue':'JH_sales'},inplace=True)
else:
    df_JH=pd.DataFrame()

if 'BadHabit' in x:
    df_BadHabit=daily_df1.groupby('sub_channel').get_group('BadHabit')
    df_BadHabit=df_BadHabit['gross_revenue'].resample('y').sum()
    df_BadHabit=df_BadHabit.to_frame()
    df_BadHabit.rename(columns={'gross_revenue':'Badhabit_sales'},inplace=True)
else:
    df_BadHabit=pd.DataFrame()

if 'REM' in x:
    df_REM=daily_df1.groupby('sub_channel').get_group('REM')
    df_REM=df_REM['gross_revenue'].resample('y').sum()
    df_REM=df_REM.to_frame()
    df_REM.rename(columns={'gross_revenue':'REM_sales'},inplace=True)
else:
    df_REM=pd.DataFrame()


In [228]:
if df_PL.empty:
    df_PL['PL_sales']=np.NAN
    df_PL['sales_date']=df_axes['sales_date']
    df_PL.set_index('sales_date',inplace=True)

    
else:pass

if df_US.empty:
    df_US['US_sales']=np.NAN
    df_US['sales_date']=df_axes['sales_date']
    df_US.set_index('sales_date',inplace=True)

else:pass

if df_BadHabit.empty:
    df_BadHabit['BadHabit_sales']=np.NAN
    df_BadHabit['sales_date']=df_axes['sales_date']
    df_BadHabit.set_index('sales_date',inplace=True)

else:pass

if df_REM.empty:
    df_REM['REM_sales']=np.NAN
    df_REM['sales_date']=df_axes['sales_date']
    df_REM.set_index('sales_date',inplace=True)
else:pass

if df_JH.empty:
    df_JH['JH_sales']=np.NAN
    df_JH['sales_date']=df_axes['sales_date']
    df_JH.set_index('sales_date',inplace=True)
else:pass

In [229]:
try:
    df_extended=pd.merge(df_BadHabit,df_REM,on='sales_date',how='outer')
    df_extended=pd.merge(df_extended,df_JH,on='sales_date',how='outer')
    df_extended=pd.merge(df_extended,df_PL,on='sales_date',how='outer')
    df_extended=pd.merge(df_extended,df_US,on='sales_date',how='outer')
except:
    print(traceback.print.exc())
    
else:
    df_extended["sum"] = df_extended.sum(axis=1)
    df_percentage=df_extended.copy()

In [230]:
if 'BadHabit_sales' in df_percentage.columns:
    df_percentage['BadHabit_sales']=(df_percentage['BadHabit_sales']/df_percentage['sum'])*100
if 'REM_sales' in df_percentage.columns:
    df_percentage['REM_sales']=(df_percentage['REM_sales']/df_percentage['sum'])*100
if 'JH_sales' in df_percentage.columns:
    df_percentage['JH_sales']=(df_percentage['JH_sales']/df_percentage['sum'])*100
if 'PL_sales' in df_percentage.columns:
    df_percentage['PL_sales']=(df_percentage['PL_sales']/df_percentage['sum'])*100
if 'US_sales' in df_percentage.columns:
    df_percentage['US_sales']=(df_percentage['US_sales']/df_percentage['sum'])*100
    
df_percentage['sum']=(df_percentage['sum']/df_percentage['sum'])*100
    


In [231]:
df_percentage.replace(np.NaN,0,inplace=True)
display(f"Monthly sales for sku number {sku_number} is :", df_percentage)

'Monthly sales for sku number 100016 is :'

,BadHabit_sales,REM_sales,JH_sales,PL_sales,US_sales,sum
sales_date,,,,,,
2017-12-31,0.0,0.0,0.0,0.000000,0.000000,0.0
2018-12-31,0.0,0.0,0.0,0.000000,0.000000,0.0
2019-12-31,0.0,0.0,0.0,0.000000,0.000000,0.0
2020-12-31,0.0,0.0,0.0,93.962306,6.037694,100.0
2021-12-31,0.0,0.0,0.0,90.200304,9.799696,100.0


In [232]:
df_percentage.to_csv(fr'/home/ec2-user/SageMaker/Pratik/Daily Forecast/{sku_number} sku sub channel proportion by year.csv')

In [ ]:
# if 'US' in x:
#     df_US=daily_df1.groupby('sub_channel').get_group('US')
#     df_US=df_US['gross_revenue'].resample('y').sum()
#     df_US=df_US.to_frame()
#     df_US.rename(columns={'gross_revenue':'US_sales'},inplace=True)
# else:
#     df_US=pd.DataFrame()
# if 'PL' in x:
#     df_PL=daily_df1.groupby('sub_channel').get_group('PL')
#     df_PL=df_PL['gross_revenue'].resample('y').sum()
#     df_PL=df_PL.to_frame()
#     df_PL.rename(columns={'gross_revenue':'PL_sales'},inplace=True)
# else:
#     df_PL=pd.DataFrame()
# if 'JH' in x:
#     df_JH=daily_df1.groupby('sub_channel').get_group('JH')
#     df_JH=df_JH['gross_revenue'].resample('y').sum()
#     df_JH=df_JH.to_frame()
#     df_JH.rename(columns={'gross_revenue':'JH_sales'},inplace=True)
# else:
#     df_JH=pd.DataFrame()
# if 'BadHabit' in x:
#     df_BadHabit=daily_df1.groupby('sub_channel').get_group('BadHabit')
#     df_BadHabit=df_BadHabit['gross_revenue'].resample('y').sum()
#     df_BadHabit=df_BadHabit.to_frame()
#     df_BadHabit.rename(columns={'gross_revenue':'BadHabit_sales'},inplace=True)
# else:
#     df_BadHabit=pd.DataFrame()
# if 'REM' in x:
#     df_REM=daily_df1.groupby('sub_channel').get_group('REM')
#     df_REM=df_REM['gross_revenue'].resample('y').sum()
#     df_REM=df_REM.to_frame()
#     df_REM.rename(columns={'gross_revenue':'REM_sales'},inplace=True)
# else:
#     df_REM=pd.DataFrame()

In [ ]:
# if df_PL.empty:
#     df_PL['PL_sales']=np.NAN
# else:pass

# if df_US.empty:
#     df_US['US_sales']=np.NAN
# else:pass

# if df_BadHabit.empty:
#     df_BadHabit['BadHabit_sales']=np.NAN
# else:pass

# if df_REM.empty:
#     df_REM['REM_sales']=np.NAN
# else:pass

# if df_JH.empty:
#     df_JH['JH_sales']=np.NAN
# else:pass

In [ ]:
# try:
#     df_extended=pd.merge(df_BadHabit,df_REM,on='sales_date',how='outer')
#     df_extended=pd.merge(df_extended,df_JH,on='sales_date',how='outer')
#     df_extended=pd.merge(df_extended,df_PL,on='sales_date',how='outer')
#     df_extended=pd.merge(df_extended,df_US,on='sales_date',how='outer')
# except:
#     print(traceback.print.exc())

In [ ]:
# df_extended["sum"] = df_extended.sum(axis=1)
# df_extended

In [ ]:
# df_percentage=df_extended.copy()

In [ ]:
# if 'BadHabit_sales' in df_percentage.columns:
#     df_percentage['BadHabit_sales']=(df_percentage['BadHabit_sales']/df_percentage['sum'])*100
# if 'REM_sales' in df_percentage.columns:
#     df_percentage['REM_sales']=(df_percentage['REM_sales']/df_percentage['sum'])*100
# if 'JH_sales' in df_percentage.columns:
#     df_percentage['JH_sales']=(df_percentage['JH_sales']/df_percentage['sum'])*100
# if 'PL_sales' in df_percentage.columns:
#     df_percentage['PL_sales']=(df_percentage['PL_sales']/df_percentage['sum'])*100
# if 'US_sales' in df_percentage.columns:
#     df_percentage['US_sales']=(df_percentage['US_sales']/df_percentage['sum'])*100
    
# df_percentage['sum']=(df_percentage['sum']/df_percentage['sum'])*100

In [ ]:
# display(f"Yearly sales for sku number {sku_number} is :", df_percentage)